# Suscripción de Azure Quantum
### Conexión al espacio de trabajo de Azure Quantum

In [1]:
%azure.connect "/subscriptions/b3ed2c3e-e784-4a4e-9474-a3d279d2f9e4/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/LaboratorioQuantum-Ximena-Toledo-RIvera" location="eastus"


Authenticated using Microsoft.Azure.Quantum.Authentication.TokenFileCredential


Connected to Azure Quantum workspace LaboratorioQuantum-Ximena-Toledo-RIvera in location eastus.


Target ID,Current Availability,Average Queue Time (Seconds)
microsoft.estimator,Available,0
rigetti.sim.qvm,Available,5
rigetti.qpu.aspen-m-2,Available,5
rigetti.qpu.aspen-m-3,Available,5


# Implementación del Algoritmo de Shor
## Fase 1 
1. Invocación de bibliotecas necesarias.
2. Creación de un generador de números pseudoaleatorios.
    + Se inicializa un bit cuántico.
    + Se coloca en superposición.
    + Se somete al proceso de medición.
    + Se retorna el resultado.


In [2]:
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Oracles;
open Microsoft.Quantum.Random;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Characterization;

operation ramdomNumberGeneratorTest() : Result {
        use q1 = Qubit();
        Message("Estado inicial de Qubit:");
        DumpMachine();
        H(q1);
        Message("Estado de superpocision:");
        DumpMachine();
        let measuredQubit = M(q1);
        Message("Proceso de medicion:");
        DumpMachine();
        return measuredQubit;
    }


ramdomNumberGeneratorTest

In [3]:
%simulate ramdomNumberGeneratorTest

Estado inicial de Qubit:


|0⟩	1 + 0𝑖
|1⟩	0 + 0𝑖

Estado de superpocision:


|0⟩	0.7071067811865476 + 0𝑖
|1⟩	0.7071067811865476 + 0𝑖

Proceso de medicion:


|0⟩	0 + 0𝑖
|1⟩	1 + 0𝑖

One

In [4]:
operation ramdomNumberGenerator() : Result {
        use q1 = Qubit();
        H(q1);
        let measuredQubit = M(q1);
        return measuredQubit;
    }

ramdomNumberGenerator

In [5]:
%simulate ramdomNumberGenerator

One

## Generador de número pseudoaleatorio
1. Definimos un número máximo y mínimo
2. Definición de la cantidad de bits cuánticos expresados en valores enteros para llegar al valor máximo.
3. Crear cadena de longitud n.
4. Si la cadena de bits supera el valor máximo, crear cadena de longitud n.
5. Si la cadena de bits es menor al valor mínimo, crear cadena de longitd n.
6. Retornar valor aleatorio.

In [6]:
operation randomNumberRange(min: Int, max: Int) : Int {
        mutable output = 0;
        repeat { // Bucle para generar números aleatorios hasta que genere uno que sea igual o menor que el máximo o igual o mayor que el mínimo
            mutable bits = [];
            for indexBit in 1..BitSizeI(max) { // Retornar el valor de bits necesarios
                set bits = bits + [ramdomNumberGenerator()];
            }
            for indexBit in 1..BitSizeI(min) {
                set bits = bits + [ramdomNumberGenerator()];
            }
            set output = ResultArrayAsInt(bits);
        } until (output >= min and output <= max );
        Message($"Número aleatorio entre {min} y {max}: ");
        return output;
    }

randomNumberRange

In [7]:
%simulate randomNumberRange min=10 max=20

Número aleatorio entre 10 y 20: 


15

In [8]:
%simulate randomNumberRange min=100 max=200

Número aleatorio entre 100 y 200: 


163

In [9]:
%simulate randomNumberRange min=1000 max=2000

Número aleatorio entre 1000 y 2000: 


1009

In [10]:
%simulate randomNumberRange min=9999 max=19999


Número aleatorio entre 9999 y 19999: 


15244

In [12]:
%simulate randomNumberRange min=99999 max=199999

## Algoritmo de Euclides
1. Hallar el máximo común divisor para identificar los factores primos.
    + Si restamos el número menor del número mayor, el máximo común divisor no cambia.
    + Mantener restas sucesivas al número mayor.
    + Dividir el número menor
        + Detenerse cuando el residuo es 0.

## Método 1: Llamada directa a la función

In [11]:
function euclideanAlgoGCDm1(a: Int, N: Int) : Int {
        Message($"GCD de {a} y {N} es..  ");
        return GreatestCommonDivisorI((a),(N));
}

euclideanAlgoGCDm1

In [12]:
%simulate euclideanAlgoGCDm1 a=60 N=48

GCD de 60 y 48 es..  


12

## Método 2: Versión recursiva

In [13]:
operation euclideanAlgoGCDm2(a: Int, N: Int) : Int {
        if((N) == 0) {
            return a;
        }  Message($"GCD: {a} -> "); 
        return euclideanAlgoGCDm2((N), (a) % (N));
    }

euclideanAlgoGCDm2

In [14]:
%simulate euclideanAlgoGCDm2 a=60 N=48

GCD de 60 -> 
GCD de 48 -> 


12

## Método 3: Restas sucesivas

In [47]:
operation euclideanAlgoGCDm3(a: Int, N: Int) : Int {
        if((a) == 0) {
            return (N);
        } if ((N) == 0) {
            return (a);
        } if ((a) == (N)) {
            return (a);
        } if ((a) > (N)) {
            Message($"GCD de {a} y {N}:"); 
            return euclideanAlgoGCDm3((a)-(N), (N));
            
        } 
        return euclideanAlgoGCDm3((a), (N)-(a));
    }

euclideanAlgoGCDm3

In [48]:
%simulate euclideanAlgoGCDm3 a=60 N=48

GCD de 60 y 48:


12

# Implementación del Algoritmo de Shor
## Fase 2: Parte clásica 
1. Determinar si el orden es par
    + gcd=(a,n)=1
        + a^(φ(n)) ≡ 1 (mod n)
2. Una vez realizada la condición del teorema de Euler aplicamos el teorema de euclides.
3. El valor del orden se le resta 1 y se aplica junto con el número primo para encontrar el primer factor.
4. El valor del orden se le suma 1 y se aplica junto con el número primo para encontrar el primer factor.
    + Retorna el factor y el módulo dividido entre el factor.
5. Fin

φ(n): Es equivalente al orden dividido entre dos.

In [49]:
operation possibleFactorsR (module: Int, randomGenerator: Int, r:Int) : (Bool, (Int, Int)) {
        if r % 2 == 0 {
            let halfExponentiation = ExpModI(randomGenerator, r/2, module);
            if halfExponentiation != module - 1 {
                let factor = MaxI (
                    euclideanAlgoGCDm3(halfExponentiation - 1, module),
                    euclideanAlgoGCDm3(halfExponentiation + 1, module)
                );
                return (true, (factor, module / factor));
            } else {
                return (false, (1,1));
            }
        } else {
        return(false, (1,1));
        } 
    }

possibleFactorsR

## Multiplicador de un entero modular
### Fase 3: Parte cuántica
1. Realiza la multiplicación modular por una constante entera en un registro qubit.
2. Dado un estado de un Qubi, se multiplica una constante y por el modulo de N.
3. Entonces la operación implementa una operación unitaria definida por:

|y⟩↦|(a⋅y) mod N⟩

4. Ejecutar el algoritmo de estimación de fase cuántica para el calculo entre el registro de almacenamiento y computacional.
5. Fin

In [50]:
operation applyFindingOracle (randomGenerator : Int, module: Int, power : Int, target: Qubit[] ) : Unit is Adj + Ctl{
        Fact(IsCoprimeI(randomGenerator,module),"El número aleatorio es `randomGenerator`, y el modulo es `module`.");
        // Realización de una multiplicación modular por una constante entera en un registro qubit.
        MultiplyByModularInteger(ExpModI(randomGenerator, power, module),module, LittleEndian(target));
    }

applyFindingOracle

# Función convergente de fracción continua
Encuentra la fracción continua convergente más cercana a la fracción con el denominador menor o igual al limite del denominador, además, se implementa el calculo del máximo común divisor.


In [51]:
operation periodFrequency (module : Int, frequencyEstimation : Int, bitsPrecision: Int, currentDivisor: Int) : Int {
        let (s, r) = (ContinuedFractionConvergentI(Fraction(frequencyEstimation, 2 ^ bitsPrecision), module))!;
        let (sAbsolute, rAbsolute) = (AbsI(s), AbsI(r));
        //  Calcula el máximo común divisor de dos enteros (se llama al método 3 de implementación del GCD).
        return (rAbsolute * currentDivisor) / euclideanAlgoGCDm3(currentDivisor, rAbsolute);
    } 

periodFrequency

## Estimación de la frecuencia
Esta función realiza la multiplicación de un entero modular a partir de la sumatoria del registro computacional más el registro de almacenamiento desde 0 hasta n bits.

1. Se determina los bits de precisión
2. Se crea una cadena de bits cuánticos de valores propios correspondientes a un vector.
3. Se crea una variable de registro para la codificación de valores con índice 0 (el bit más bajo de un entero sin signo).
4. Aplicación de la operación XOR al registro de los valores propios basados en el entero 1.

|y⟩→|y⊕a⟩

5. Se recorre el indicé de los bits de precisión
    + Dentro de cada valor con un bit cuántico en superposición, se alplica una rotación sobre el estado |1⟩.

R1(n,k):=diag(1,e^(iπk/2n))

6. Si el valor de la medición se encuentra en la base Z es igual a 1
    + Se reestablece al estado inicial después de la medición
    + Se asigna la frecuencia de estimación
7. Se resetean los valores propio.
8. se retorna el valor de la frecuencia.
9. Fin

  
    

In [52]:
operation EstimateFrequencyValue (randomGenerator : Int, module: Int, bits : Int) : Int {   
        mutable frequencyEstimation = 0;
        
        let bitsPrecision = 2 * bits + 1;

        use eigenStateRegister = Qubit[bits];
        
        let eigenStateRegisterLittleEndian = LittleEndian(eigenStateRegister);

        // Aplicando una operación XOR entre un entero clásico y un entero representado por un registro de qubits.
        ApplyXorInPlace(1, eigenStateRegisterLittleEndian);
        let oracle = applyFindingOracle(randomGenerator, module, _, _);

        use a = Qubit();
        for index in bitsPrecision -1..-1..0 {
            within {
                H(a);
            } apply {
                Controlled oracle([a], (1 <<< index, eigenStateRegisterLittleEndian!));
                // Aplicar una rotación sobre el estado |1> por un ángulo especificado como una fracción diádica.
                R1Frac(frequencyEstimation, bitsPrecision - 1 - index, a);
            } if MResetZ(a) == One {
                set frequencyEstimation += 1 <<< (bitsPrecision - 1 - index);
            }
        }
        // Mide los qubits y se asegura de que estén en el estado |0⟩ para que puedan liberarse de forma segura.
        ResetAll(eigenStateRegister);
        return frequencyEstimation;
    }

EstimateFrequencyValue

## Algoritmo de estimación de fase
En este apartado se identifican los eigenvalues de operadores unitarios, dado un operador unitario U y un estado |ϕ⟩ tal que |ϕ⟩ es un autoestado de U con un eigenvalue desconocido ϕ, a este problema se le conoce como estimación de fase.

U|ϕ⟩=ϕ|ϕ⟩

1. Revisar si una condición clásica es verdadera.
    + Se revisa si el número aleatorio y el módulo son coprimos entre sí.
2. Se almacena el resultado de la estimación de fase dentro de una variable muteable.
3. Si la estimación de frecuencia es diferente de 0, entonces se asigna el valor del periodo con los valores determinados.
4. Si la estimación de frecuencia es igual a 0, se notifica al usuario.
5. Se retorna el valor.
6. Fin


 

In [53]:
operation phaseEstimation (randomGenerator: Int, module: Int ) : Int {
        
        Fact(IsCoprimeI(randomGenerator,module),"El numero aleatorio es `randomGenerator` y el modulo es `module`.");

        mutable result = 1;

        let bits = BitSizeI(module);

        let bitsPrecision = 2 * bits + 1;

        mutable frequencyEstimation = 0;

        set frequencyEstimation = EstimateFrequencyValue (randomGenerator, module,bits);
        
        if  frequencyEstimation != 0 {
            set result = periodFrequency (module, frequencyEstimation, bitsPrecision, result);
        } else {
            Message("La frecuencia estimada tiene el valor 0.");
        }
        return result;
    }

phaseEstimation

## Prueba de implementación del algoritmo Shor
1. Se determina si el número es par
    + Si el número es par, se calcula n/2
2. Se establecen los factores predeterminados
3. Hasta que los valores de los factores esten establecidos, verificar si el número aleatorio y el número primo son coprimos:
    + Calcular el orden por medio de la estimación de fase.
    + Asignar los valores por defecto a los posibles factores encontrados.
4. Si no son coprimos, ejecutar el algoritmo de euclides directamente.
5. Si la condición es diferente, se bloquea y continua la iteración.
6. Se retornan los factores.
7. Fin


In [54]:
operation shorImplementatonTest (n: Int): (Int, Int) {

        if n % 2 == 0 {
            // Comprobar si hay un número par.
            Message("Caso trivial: El número es par.");
            return (n/2, 2);
        }
        
        // Configura los factores desconocidos y establece los valores predeterminados.
        mutable setUpFactors = false;
        mutable defaultFactors = (1,1); 

    
        repeat {
            let randomGenerator = randomNumberRange(2, n - 1); // DrawRandomInt 
            // Casos:  IBM(2, N-1),  MICROSOT(1 < a < N-1)
    
            if IsCoprimeI (randomGenerator,n) {
                Message($"Número aleatorio: {randomGenerator}");
                let r = phaseEstimation(randomGenerator, n);
                set (setUpFactors,defaultFactors) = possibleFactorsR(n, randomGenerator, r);
            }  else {
                let GCD = euclideanAlgoGCDm3(n,randomGenerator);
                Message($"Divisor optenido: {n}, GCD: {GCD}.");
                set setUpFactors = true;
                set defaultFactors = (GCD, n/ GCD);
            }
        } until setUpFactors 
        fixup {
        Message("La estimación del periodo no retorna un factor valido.");
        }
        return defaultFactors;
    }

shorImplementatonTest

In [56]:
%simulate shorImplementatonTest n=15

Número aleatorio entre 2 y 14: 
GCD de 15 y 10:
Divisor optenido: 15, GCD: 5.


(5, 3)

In [57]:
%simulate shorImplementatonTest n=15

Número aleatorio entre 2 y 14: 
GCD de 15 y 5:
GCD de 10 y 5:
Divisor optenido: 15, GCD: 5.


(5, 3)

In [55]:
%simulate shorImplementatonTest n=15

Número aleatorio entre 2 y 14: 
GCD de 15 y 9:
GCD de 6 y 3:
Divisor optenido: 15, GCD: 3.


(3, 5)

In [58]:
%simulate shorImplementatonTest n=21

Número aleatorio entre 2 y 20: 
GCD de 21 y 12:
GCD de 9 y 3:
GCD de 6 y 3:
Divisor optenido: 21, GCD: 3.


(3, 7)

In [59]:
%simulate shorImplementatonTest n=35

Número aleatorio entre 2 y 34: 
Número aleatorio: 33
La estimación del periodo no retorna un factor valido.
Número aleatorio entre 2 y 34: 
Número aleatorio: 31
La frecuencia estimada tiene el valor 0.
La estimación del periodo no retorna un factor valido.
Número aleatorio entre 2 y 34: 
Número aleatorio: 17
GCD de 12 y 11:
GCD de 14 y 7:


(7, 5)

In [60]:
%simulate shorImplementatonTest n=102

Caso trivial: El número es par.


(51, 2)

In [61]:
%simulate shorImplementatonTest n=1004

Caso trivial: El número es par.


(502, 2)

In [62]:
%simulate shorImplementatonTest n=39

Número aleatorio entre 2 y 38: 
Número aleatorio: 19
La estimación del periodo no retorna un factor valido.
Número aleatorio entre 2 y 38: 
Número aleatorio: 14
GCD de 15 y 9:
GCD de 6 y 3:


(13, 3)